# Launch an interview
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for simulating qualitative research interviews. In a series of steps we construct an AI agent representing a qualitative researcher and launch an interview with specified topics and interview subjects.

**Please also see examples of the `Conversation` [module](https://github.com/expectedparrot/edsl/tree/main/edsl/conversation) which automates the methods used below to simulate a conversation with multiple agents:**

* [Buying a car](https://github.com/expectedparrot/edsl/blob/main/edsl/conversation/car_buying.py)
* [Negotiating a mug](https://github.com/expectedparrot/edsl/blob/main/edsl/conversation/mug_negotiation.py)

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

In [1]:
from edsl import QuestionFreeText, Agent, Scenario, Model

import textwrap
from rich import print

EDSL works with many popular language models ([learn more](https://docs.expectedparrot.com/en/latest/language_models.html) about using models). Here we select one to use in running in our survey:

In [2]:
model = Model("gpt-4o")

Here we create methods for constructing agents representing a researcher and subject, and conducting an interview in the form of a survey that we can observe. (Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [running surveys](https://docs.expectedparrot.com/en/latest/surveys.html)).

In [3]:
def construct_subject(name, traits={}):
    return Agent(name=name, traits=traits)


def construct_researcher(product, topics):
    background = textwrap.dedent(
        f"""\
    You are an expert qualitative researcher. You are researching this product: {product}. 
    You are interested in conducting interviews to learn people's views on the following topics relating 
    to the product: {topics}."""
    )

    instruction = textwrap.dedent(
        f"""\
    You know to ask questions that are appropriate to the age and experience of an interview subject.
    You know to not ask questions that an interview subject would not be able to answer, 
    e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
    You ask excellent follow-up questions.    
    """
    )
    return Agent(traits={"background": background}, instruction=instruction)


def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario(
        {"subject": str(subject.traits), "dialog_so_far": dialog_so_far}
    )
    meta_q = QuestionFreeText(
        question_name="next_question",
        question_text="""
        These are the biographic details of the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """,
    )
    question_text = (
        meta_q.by(model)
        .by(researcher)
        .by(scenario)
        .run()
        .select("next_question")
        .first()
    )
    return question_text


def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name="question",
        question_text=f"""
        This is your current dialog with the interview subject: {dialog_so_far}. 
        You are now being asked:"""
        + question_text,
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first()
    return response


def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)
    return {"question": question_text, "response": response}


def dialog_to_string(d):
    return "\n".join(
        [f"Question: {d['question']}\nResponse: {d['response']}" for d in d]
    )

In [5]:
subject = construct_subject(name="Alice", traits={"gender": "female", "age": 12})
researcher = construct_researcher(product="cassette tapes", topics="technology, music")
result = ask_question(subject, researcher, "")
print(result)

{
    'question': 'What are some of your favorite ways to listen to music, and have you ever heard of or used a 
cassette tape before?',
    'response': "I love listening to music on my phone with my earbuds, especially when I'm doing homework or just 
chilling. Sometimes, I also use a Bluetooth speaker when I’m hanging out with friends. As for cassette tapes, I’ve 
heard of them! My parents have talked about them, and I think they even have some old ones stored away. I’ve never 
used one myself, though. They sound kind of cool and retro!"
}

In [6]:
dialog_so_far = []
for i in range(5):
    result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
    print(result)
    dialog_so_far.append(result)
print(dialog_so_far)

{
    'question': 'What are some of your favorite ways to listen to music, and have you ever heard of or used a 
cassette tape before?',
    'response': "I love listening to music on my phone with my earbuds, especially when I'm doing homework or just 
chilling. Sometimes, I also use a Bluetooth speaker when I’m hanging out with friends. As for cassette tapes, I’ve 
heard of them! My parents have talked about them, and I think they even have some old ones stored away. I’ve never 
used one myself, though. They sound kind of cool and retro!"
}

{
    'question': "That's interesting that you've heard about cassette tapes from your parents! Can you tell me more 
about what they've shared with you about using cassette tapes? Do you think you'd be interested in trying one out 
if you had the chance?",
    'response': 'My parents said that cassette tapes were the main way they listened to music when they were 
younger. They had to use a tape player, and sometimes they would make mixtapes by recording songs from the radio, 
which sounds really fun! They also mentioned how you had to rewind or fast forward to find the song you wanted, 
which seems kind of tricky compared to just tapping on a screen now. I think it would be cool to try one out, just 
to see what it was like back then. It’s like experiencing a little piece of history! Plus, I love the idea of 
making my own mixtape.'
}

{
    'question': "You've mentioned that making mixtapes sounds really fun and like a piece of history. If you could 
create a mixtape today, what songs or artists would you include, and why?",
    'response': "If I could make a mixtape today, I'd definitely include some of my favorite songs and artists that
I love listening to right now. I'd probably start with some Taylor Swift because her songs are so catchy and 
relatable. Then, I'd add some Olivia Rodrigo because her music is full of emotions and I love singing along. I'd 
also include a few tracks from Imagine Dragons because their music is super energetic and great for getting pumped 
up. Maybe I'd throw in some Billie Eilish for those more chill, moody vibes. Oh, and I can't forget about BTS, 
since their songs are really fun and they always make me want to dance! I think having a mix of different styles 
and moods would make the mixtape really interesting and fun to listen to anytime."
}

{
    'question': "Those are some great choices for your mixtape! It sounds like you have a diverse taste in music. 
I'm curious, since you mentioned different styles and moods, how do you usually decide what type of music to listen
to at different times? Does your mood influence your music choices, or do you have specific times or activities 
when you prefer certain artists or genres?",
    'response': "Thanks! Yeah, my mood definitely influences what music I listen to. Like, if I'm feeling super 
happy or need some energy, I'll go for upbeat songs like Imagine Dragons or BTS. Their music always gets me moving!
If I'm feeling a bit down or just want to relax, I'll listen to something more chill, like Billie Eilish. When I'm 
doing homework or need to focus, I usually pick songs that aren't too distracting, maybe some instrumental stuff or
just softer tunes. And if I'm just hanging out with friends, we might put on something fun and catchy like Taylor 
Swift or Olivia Rodrigo. It really depends on what I'm doing and how I'm feeling!"
}

{
    'question': "It's great to hear how music plays such an important role in your daily life and how you choose 
songs based on your mood and activities. Since you mentioned that your parents have shared stories about cassette 
tapes and making mixtapes, I'm curious about how you share music with your friends. Do you have any favorite ways 
to share your favorite songs or playlists with them, and how do you think that compares to the idea of making and 
sharing a mixtape on a cassette tape?",
    'response': "I love sharing music with my friends! Usually, I just send them links to songs or playlists 
through apps like Spotify or Apple Music. It's super easy and quick, and they can listen to the songs right away. 
Sometimes, we even make collaborative playlists where everyone adds their favorite tracks, which is really fun 
because you get to discover new music from each other."
}

[
    {
        'question': 'What are some of your favorite ways to listen to music, and have you ever heard of or used a 
cassette tape before?',
        'response': "I love listening to music on my phone with my earbuds, especially when I'm doing homework or 
just chilling. Sometimes, I also use a Bluetooth speaker when I’m hanging out with friends. As for cassette tapes, 
I’ve heard of them! My parents have talked about them, and I think they even have some old ones stored away. I’ve 
never used one myself, though. They sound kind of cool and retro!"
    },
    {
        'question': "That's interesting that you've heard about cassette tapes from your parents! Can you tell me 
more about what they've shared with you about using cassette tapes? Do you think you'd be interested in trying one 
out if you had the chance?",
        'response': 'My parents said that cassette tapes were the main way they listened to music when they were 
younger. They had to use a tape player, and sometimes they would make mixtapes by recording songs from the radio, 
which sounds really fun! They also mentioned how you had to rewind or fast forward to find the song you wanted, 
which seems kind of tricky compared to just tapping on a screen now. I think it would be cool to try one out, just 
to see what it was like back then. It’s like experiencing a little piece of history! Plus, I love the idea of 
making my own mixtape.'
    },
    {
        'question': "You've mentioned that making mixtapes sounds really fun and like a piece of history. If you 
could create a mixtape today, what songs or artists would you include, and why?",
        'response': "If I could make a mixtape today, I'd definitely include some of my favorite songs and artists 
that I love listening to right now. I'd probably start with some Taylor Swift because her songs are so catchy and 
relatable. Then, I'd add some Olivia Rodrigo because her music is full of emotions and I love singing along. I'd 
also include a few tracks from Imagine Dragons because their music is super energetic and great for getting pumped 
up. Maybe I'd throw in some Billie Eilish for those more chill, moody vibes. Oh, and I can't forget about BTS, 
since their songs are really fun and they always make me want to dance! I think having a mix of different styles 
and moods would make the mixtape really interesting and fun to listen to anytime."
    },
    {
        'question': "Those are some great choices for your mixtape! It sounds like you have a diverse taste in 
music. I'm curious, since you mentioned different styles and moods, how do you usually decide what type of music to
listen to at different times? Does your mood influence your music choices, or do you have specific times or 
activities when you prefer certain artists or genres?",
        'response': "Thanks! Yeah, my mood definitely influences what music I listen to. Like, if I'm feeling super
happy or need some energy, I'll go for upbeat songs like Imagine Dragons or BTS. Their music always gets me moving!
If I'm feeling a bit down or just want to relax, I'll listen to something more chill, like Billie Eilish. When I'm 
doing homework or need to focus, I usually pick songs that aren't too distracting, maybe some instrumental stuff or
just softer tunes. And if I'm just hanging out with friends, we might put on something fun and catchy like Taylor 
Swift or Olivia Rodrigo. It really depends on what I'm doing and how I'm feeling!"
    },
    {
        'question': "It's great to hear how music plays such an important role in your daily life and how you 
choose songs based on your mood and activities. Since you mentioned that your parents have shared stories about 
cassette tapes and making mixtapes, I'm curious about how you share music with your friends. Do you have any 
favorite ways to share your favorite songs or playlists with them, and how do you think that compares to the idea 
of making and sharing a mixtape on a cassette tape?",
        're